In [1]:
from objects import EnvironmentManager
from helpers import *

env = EnvironmentManager()
env.start_vehicles()
env.route.load_all_pax()

observation, reward, terminated, truncated, info = env.step(action=None)
while not terminated:
    action = get_action('RA')
    observation, reward, terminated, truncated, info = env.step(action=action)

history = env.get_history()
# for key in history:
#     history[key]['scenario'] = scenario
#     history[key]['episode'] = i
#     results[key].append(history[key])